In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, Conv1D, MaxPooling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow import keras, config
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("GPUs Available: ", len(config.list_physical_devices('GPU')))

In [ ]:
from image_preprocessors import add_zeropad_random, add_noise_random, add_noise
# Global settings
TEST_SPLIT_SIZE = 0.90
VALIDATION_SPLIT_SIZE = 0.90
IMAGE_TARGET_SIZE = (5, 1400, 1)
BATCH_SIZE =  128
LETTER_END_POSITION = "P1"
IMAGE_PREPOCESSORS = [
    # {"func" :add_zeropad_random, "params": [-10, 15]}, 
    {"func": add_noise_random, "params": [0, 30]}
]

In [ ]:

%load_ext autoreload
%autoreload 2

from morse_helpers import create_sets
from morse_label_funcs import velocity_regression


def get_sets():
    return [
   
        # ["./training_data/MorseTrainSet_17/GEN17_VER_000/", 'wordsMatrices_17_000', "Words_17_000.csv"],
        ["./training_data/MorseTrainSet_17/GEN17_VER_100/", 'wordsMatrices_17_100', "Words_17_100.csv"],
        ["./training_data/MorseTrainSet_17/GEN17_VER_200/", 'wordsMatrices_17_200', "Words_17_200.csv"],
        # ["./training_data/MorseTrainSet_17/GEN17_VER_210/", 'wordsMatrices_17_210', "Words_17_210.csv"],
        # ["./training_data/MorseTrainSet_17/GEN17_VER_220/", 'wordsMatrices_17_220', "Words_17_220.csv"],

        ["./training_data/MorseTrainSet_17/GEN17_VER_001/", 'wordsMatrices_17_001', "Words_17_001.csv"],
        ["./training_data/MorseTrainSet_17/GEN17_VER_002/", 'wordsMatrices_17_002', "Words_17_002.csv"],
        ["./training_data/MorseTrainSet_17/GEN17_VER_011/", 'wordsMatrices_17_011', "Words_17_011.csv"],
        ["./training_data/MorseTrainSet_17/GEN17_VER_012/", 'wordsMatrices_17_012', "Words_17_012.csv"],
        # ["./training_data/MorseTrainSet_17/GEN17_VER_021/", 'wordsMatrices_17_021', "Words_17_021.csv"],
        # ["./training_data/MorseTrainSet_17/GEN17_VER_022/", 'wordsMatrices_17_022', "Words_17_022.csv"],
        ["./training_data/MorseTrainSet_17/GEN17_VER_111/", 'wordsMatrices_17_111', "Words_17_111.csv"],
        # ["./training_data/MorseTrainSet_17/GEN17_VER_222/", 'wordsMatrices_17_222', "Words_17_222.csv"],

    ]

(image_fnames, morse_labels) = create_sets(
    get_sets(), 
    IMAGE_TARGET_SIZE,
    [velocity_regression],
    letter_n=LETTER_END_POSITION,
    overwrite_images=False
)

In [ ]:
morse_labels = np.array(morse_labels[0])

In [ ]:
from morse_helpers import create_all_sets

train, labels, train_validation, labels_validation, train_test, labels_test = create_all_sets(
    image_fnames, morse_labels, TEST_SPLIT_SIZE, VALIDATION_SPLIT_SIZE, shuffle_before_test_split=True)


In [ ]:
from morse_helpers import Image_Generator, return_label_positions

training_batch_generator = Image_Generator(train, labels, BATCH_SIZE, IMAGE_TARGET_SIZE, IMAGE_PREPOCESSORS, return_label_positions)
validation_batch_generator = Image_Generator(train_validation, labels_validation, BATCH_SIZE, IMAGE_TARGET_SIZE, IMAGE_PREPOCESSORS, return_label_positions)


In [ ]:
def conv_model_velocity(input_layer):
    x = keras.layers.Cropping2D(cropping=((0, 0), (0,200)), data_format=None)(input_layer)

    x = Conv2D(90,(1,7), padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(2,1),padding="same")(x)

    x = Conv2D(90,(1,7),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(1,5),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Conv2D(90,(3,3),padding="same",activation="relu")(x)
    x = MaxPooling2D(pool_size=(1,2),padding="same")(x)

    x = Flatten()(x)

    return x

In [ ]:
from tensorflow.keras import layers

input_layer    = Input(shape=IMAGE_TARGET_SIZE)
conv_model_position_flattened = conv_model_velocity(input_layer)
output_layer_position    = Dense(1, name="regr")(conv_model_position_flattened)

model           = Model(inputs=input_layer, outputs=output_layer_position)
model.compile(loss=["mse"], optimizer='adam', metrics=["mean_absolute_error"])

print(model.summary())

In [ ]:
init_epoch = 0

In [ ]:

num_epochs = 10

def fit_model(epochs):
	
	global init_epoch
	history = model.fit(
					   training_batch_generator,
	                   steps_per_epoch = int(len(train) // BATCH_SIZE),
	                   epochs = epochs + init_epoch,
					   initial_epoch=init_epoch,
	                   verbose =1,
	                   validation_data = validation_batch_generator,
	                   validation_steps = int(len(train_validation) // BATCH_SIZE))
	
	
	init_epoch += epochs
	return history

history = fit_model(num_epochs)


In [ ]:
def show_image(img):
    plt.figure(figsize=(30,5))
    plt.xlim(0, 300)
    plt.imshow(img)
    plt.show()

In [ ]:
tempo_diff_above = 1

def get_deviating_predictions(tempo_diff_above, generator, predictions):
    regression_differences = []
    indexer = 0

    for imgs_batch, labels_batch in generator:

        for i in range(len(imgs_batch)):

            regr_pred = predictions[indexer] * 100
            regr_test_label = labels_batch[i] * 100

            if abs(regr_pred[0] - regr_test_label) > tempo_diff_above:
                regression_differences.append([regr_pred, regr_test_label, imgs_batch[i]])

            indexer += 1

    return regression_differences

In [ ]:
# TODO : print model summary, model history, image optimizers, training sets used, plot graphs, save to pdf

def print_noise_results(predictions, noise_level, tempo_diff_above, differences, evaluations):
    print("Total predictions:", len(predictions))
    print("Noise level:", noise_level)
    print("Total velocity predictions off by more than n:", tempo_diff_above, ":", len(differences))
    print("Velocity predictions percentage incorrect:", round( (len(differences) / len(predictions) * 100), 4), "%")
    print("Model evaluation:", evaluations)
    print("------------------------------------------------------")

In [ ]:
%%capture cap --no-stderr
import inspect

# TODO : create json report instead

print("\nData sets used:")
print(inspect.getsource(get_sets))

print("\nData size:")
print("Training set size:", len(train))
print("Validation set size:", len(train_validation))
print("Test set size:", len(train_test))

print("\nImage pre-processors used:")
print(IMAGE_PREPOCESSORS)

print("\nConvolution layer:")
print(inspect.getsource(conv_model_velocity))

print("\nModel summary:")
print(model.summary())
print("\nOptimizer:", model.optimizer)

print("\nModel performance:")
print(history.history)
print("\nFinal epoch performance:")
for key in history.history.keys():
    print(key, history.history[key][-1])

print("\nTotal epochs:", init_epoch)

print("\nResults:")

noise_levels = [0.2]
for noise_level in noise_levels:

    image_preprocessors_test = [{"func": add_noise, "params": noise_level}]

    test_batch_generator = Image_Generator(train_test, labels_test, BATCH_SIZE, IMAGE_TARGET_SIZE, image_preprocessors_test, return_label_positions)
    predictions = model.predict(test_batch_generator)
    regression_differences  = get_deviating_predictions(tempo_diff_above, test_batch_generator, predictions)
    evaluations = model.evaluate(test_batch_generator, verbose = 0)

    print_noise_results(predictions, noise_level, tempo_diff_above, regression_differences, evaluations)

In [ ]:
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
with open('results_velocity/results_velocity.txt' + timestr, 'w') as f:
    f.write(cap.stdout)

In [ ]:

counter = 0   
for diff in regression_differences:

    counter += 1
    if counter > 100:
        break

    pred, correct, img = diff

    img_pred = img.copy()
    img_correct = img.copy()

    print('Prediction', round(pred[0]))
    show_image(img_pred)

    print('Correct', round(correct))
    show_image(img_correct)

    print("----------------------------------------------------------------------------------------")
